<a href="https://colab.research.google.com/github/FrancescaNegriUniMiB/focsproject/blob/main/Foundations_CS_2425.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FOUNDATIONS OF COMPUTER SCIENCE - PROJECT FOR A.Y. 2024-2025

### Project text

1. Extract all trips with `trip_distance` larger than 50
2. Extract all trips where `payment_type` is missing
3. For each (`PULocationID`, `DOLocationID`) pair, determine the number of trips
4. Save all rows with missing `VendorID`, `passenger_count`, `store_and_fwd_flag`, `payment_type` in a new DataFrame called `bad`, and remove those rows from the original DataFrame
5. Add a `duration` column storing how long each trip has taken (use `tpep_pickup_datetime`, `tpep_dropoff_datetime`)
6. For each pickup location, determine how many trips have started there
7. Cluster the pickup time of the day into 30-minute intervals (e.g., from 02:00 to 02:30)
8. For each interval, determine the average number of passengers and the average fare amount
9. For each payment type and each interval, determine the average fare amount
10. For each payment type, determine the interval when the average fare amount is maximum
11. For each payment type, determine the interval when the overall ratio between the tip and the fare amounts is maximum
12. Find the location with the highest average fare amount
13. Build a new DataFrame (called `common`) where, for each pickup location, we keep all trips to the 5 most common destinations (i.e., each pickup location can have different common destinations)
14. On the `common` DataFrame, for each payment type and each interval, determine the average fare amount
15. Compute the difference of the average fare amount computed in the previous point with those computed at point 9
16. Compute the ratio between the differences computed in the previous point and those computed in point 9  
    **Note:** You have to compute a ratio for each pair (`payment type`, `interval`)
17. Build chains of trips. Two trips are consecutive in a chain if:
    1. They have the same `VendorID`
    2. The pickup location of the second trip is also the dropoff location of the first trip
    3. The pickup time of the second trip is after the dropoff time of the first trip
    4. The pickup time of the second trip is at most 2 minutes later than the dropoff time of the first trip

## Importing


In [1]:
import pandas as pd
import time as t
import numpy as np
from bisect import bisect_left, bisect_right
from collections import defaultdict, deque

if 'executed' not in globals():

  executed = True

  !pip install -q gdown
  !gdown --fuzzy https://drive.google.com/file/d/1IUOdTOYgjco0ggTVsNluQOl-xPbMZ3Z-/view?usp=sharing

  df = pd.read_csv('/content/focs_data.csv',dtype={'store_and_fwd_flag':object})
  display(df.head())

  df_backup = df
else:
  df = df_backup
  print("Reset df: ok.")

Downloading...
From (original): https://drive.google.com/uc?id=1IUOdTOYgjco0ggTVsNluQOl-xPbMZ3Z-
From (redirected): https://drive.google.com/uc?id=1IUOdTOYgjco0ggTVsNluQOl-xPbMZ3Z-&confirm=t&uuid=fe2e9cc1-de77-4421-a87a-703cca8743a7
To: /content/focs_data.csv
100% 594M/594M [00:06<00:00, 90.0MB/s]


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.2,1.0,N,238,239,1.0,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5
1,1.0,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.2,1.0,N,239,238,1.0,7.0,3.0,0.5,1.50,0.0,0.3,12.30,2.5
2,1.0,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.6,1.0,N,238,238,1.0,6.0,3.0,0.5,1.00,0.0,0.3,10.80,2.5
3,1.0,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.8,1.0,N,238,151,1.0,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0
4,2.0,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.0,1.0,N,193,193,2.0,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.0


**PRE-PROCESSING**
The following rows are being removed because they can't represent a valid trip.
A valid trip is assumed to have these columns always positive:


*   fare_amount: can be >0 or 0 (free trip or payment voided?), but not below 0
*   tip_amount: logically can't be below 0 (but could be 0)
*   trip_distance: can't be <=0 because some distance has to be covered
*   total_amount: since it's a sum of multiple components, can't be <= 0

In [2]:
cols_to_check = ['fare_amount', 'tip_amount', 'trip_distance', 'total_amount']

len_before = len(df)

for col in cols_to_check:
    n_neg = (df[col] < 0).sum()
    print(f"{col}: {n_neg} negative values found")

df = df[(df['fare_amount'] >= 0) & (df['trip_distance'] > 0) & (df['tip_amount'] >= 0) & (df['total_amount'] > 0)]


print(f"\nA total of {len_before - len(df)} rows removed.")

fare_amount: 19505 negative values found
tip_amount: 170 negative values found
trip_distance: 2338 negative values found
total_amount: 19505 negative values found

A total of 87985 rows removed.


## Task 1

Extract all trips with trip_distance larger than 50

In [3]:
trips_g_50 = df[df['trip_distance'] > 50].copy().reset_index(drop=True)
print(f"\n{len(trips_g_50)} rows found.\n")
print("Details for trip_distance greater than 50:\n")
display(trips_g_50.trip_distance.describe())


376 rows found.

Details for trip_distance greater than 50:



,trip_distance
count,376.000000
mean,624.724441
std,10838.941376
min,50.050000
25%,53.192500
50%,58.700000
75%,67.660000
max,210240.070000


## Task 2

Extract all trips where payment_type is missing

In [4]:
trips_nan_pt = df[df['payment_type'].isna()].copy().reset_index(drop=True)
print(f"\n{len(trips_nan_pt)} rows found.\n")
print("First 5 rows of the subdf:\n")
display(trips_nan_pt.head().round(2))


62679 rows found.

First 5 rows of the subdf:



,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,NaN,2020-01-01 08:51:00,2020-01-01 09:19:00,NaN,13.69,NaN,NaN,136,232,NaN,51.05,2.75,0.5,0.0,0.0,0.3,54.60,0.0
1,NaN,2020-01-01 08:38:43,2020-01-01 08:51:08,NaN,3.42,NaN,NaN,121,9,NaN,27.06,2.75,0.0,0.0,0.0,0.3,30.11,0.0
2,NaN,2020-01-01 08:27:00,2020-01-01 08:32:00,NaN,2.20,NaN,NaN,197,216,NaN,24.36,2.75,0.5,0.0,0.0,0.3,27.91,0.0
3,NaN,2020-01-01 08:46:00,2020-01-01 08:57:00,NaN,0.84,NaN,NaN,262,236,NaN,26.08,2.75,0.5,0.0,0.0,0.3,29.63,0.0
4,NaN,2020-01-01 08:21:00,2020-01-01 08:38:00,NaN,7.24,NaN,NaN,45,142,NaN,25.28,2.75,0.5,0.0,0.0,0.3,28.83,0.0


In [5]:
#print(f"Removing {len(df[df['VendorID'].isna()])} rows")
#df = df[df['VendorID'].notna()]

## Task 3

For each (PULocationID, DOLocationID) pair, determine the number of trips

In [6]:
# prompt: For each (PULocationID, DOLocationID) pair, determine the number of trips

trip_counts = df[['PULocationID','DOLocationID']].value_counts().reset_index(name='total_trips_per_route')
print(f"{len(trip_counts)} pairs found.")
print("\nStatistical data about 'total_trips_per_route' distribution:")
display(trip_counts.total_trips_per_route.describe().astype(int))

29663 pairs found.

Statistical data about 'total_trips_per_route' distribution:


,total_trips_per_route
count,29663
mean,212
std,1078
min,1
25%,2
50%,5
75%,24
max,45442


## Task 4

Save all rows with missing VendorID, passenger_count, store_and_fwd_flag, payment_type in a new DataFrame called bad, and remove those rows from the original DataFrame

In [7]:
nan_cols = ['VendorID', 'passenger_count', 'store_and_fwd_flag', 'payment_type']

null_indices = df[nan_cols].isna().any(axis=1)

print(f"Number of rows where all specified columns are null: {null_indices.sum()}")
print("\nCheck if rows null in one column are null in others:")
for col in nan_cols:
    is_null_in_col = df[col].isna()
    # Check if all rows that are null in 'col' are also null in all other columns
    check = df.loc[is_null_in_col, [c for c in nan_cols if c != col]].isna().all(axis=1)
    print(f"  - Rows where '{col}' is null are also null in all other specified columns: {check.all()}")

#If the value can be missing in just 1 of the 4 columns:
bad = df[null_indices].copy().reset_index(drop=True)
df = df.dropna(subset=nan_cols).reset_index(drop=True)

print(f"\n{len(bad)} rows removed from the original dataset.")
print(f"\nNow the dataset is {len(df)} rows long.")

Number of rows where all specified columns are null: 62679

Check if rows null in one column are null in others:
  - Rows where 'VendorID' is null are also null in all other specified columns: True
  - Rows where 'passenger_count' is null are also null in all other specified columns: True
  - Rows where 'store_and_fwd_flag' is null are also null in all other specified columns: True
  - Rows where 'payment_type' is null are also null in all other specified columns: True

62679 rows removed from the original dataset.

Now the dataset is 6254344 rows long.


## Task 5

Add a duration column storing how long each trip has taken (use tpep_pickup_datetime, tpep_dropoff_datetime)

In [8]:
start = t.time()

df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

df['duration_sec'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds().astype(int)

df['duration_str'] = df['duration_sec'].apply(lambda sec: f"{sec // 86400} days " if sec // 86400 > 0 else "") + \
                                df['duration_sec'].apply(lambda sec: f"{sec // 3600 % 24} hours " if sec // 3600 > 0 else "") + \
                                df['duration_sec'].apply(lambda sec: f"{sec // 60 % 60} minutes ") + \
                                df['duration_sec'].apply(lambda sec: f"{sec % 60} seconds")
df['duration_str'] = df['duration_str'].str.strip()

maxdur = df.loc[df['duration_sec'].idxmax()]
mindur = df.loc[df['duration_sec'].idxmin()]
print(f"\nLongest trip: {maxdur.duration_str}")
print(f"Shortest trip: {mindur.duration_str}")

end = t.time()
print(f"\n\n\n(Execution time: {round(end - start,2)} seconds.)")


Longest trip: 5 days 22 hours 5 minutes 7 seconds
Shortest trip: 49 minutes 38 seconds



(Execution time: 18.74 seconds.)


## Task 6

For each pickup location, determine how many trips have started there

In [9]:
pickup_counts = df['PULocationID'].value_counts().reset_index()
pickup_counts.columns = ['PULocationID', 'number_of_trips']

print("Top 5 most common PULocationID (with the number of trips started there):\n")
display(pickup_counts.head())
print(f"\nTotal {len(pickup_counts)} unique pickup locations.")

print("\nStatistical data about the number of trips per pickup location:")
display(pickup_counts.number_of_trips.describe().astype(int))

Top 5 most common PULocationID (with the number of trips started there):



,PULocationID,number_of_trips
0,237,290756
1,161,279430
2,236,270534
3,162,233265
4,186,226447



Total 256 unique pickup locations.

Statistical data about the number of trips per pickup location:


,number_of_trips
count,256
mean,24431
std,56871
min,1
25%,48
50%,157
75%,5282
max,290756


## Task 7

Cluster the pickup time of the day into 30-minute intervals (e.g., from 02:00 to 02:30)

In [10]:
start = t.time()

df['hour']   = df['hour2']   = df['tpep_pickup_datetime'].dt.hour
df['minute'] = df['minute2'] = df['tpep_pickup_datetime'].dt.floor('30min').dt.minute

df['hour2'] = np.where(df['minute'] == 30, (df['hour'] + 1) % 24, df['hour']) #[0-23] % 24 -> 0, 24 % 24 -> 1
df['minute2'] = np.where(df['minute'] == 30, 0, 30)

df['tpep_pickup_time_interval'] = (
    df['hour'].map('{:02}'.format) + ':' + df['minute'].map('{:02}'.format) + '-' +
    df['hour2'].map('{:02}'.format) + ':' + df['minute2'].map('{:02}'.format)
)

df.drop(columns=['hour','hour2','minute','minute2'],inplace=True)

print('\nNew column preview:')
display(df[['tpep_pickup_datetime','tpep_pickup_time_interval']].head())

print('\nList of all the possible intervals:')
display(df['tpep_pickup_time_interval'].unique())

end = t.time()
print(f"\n\n\n(Execution time: {round(end - start,2)} seconds.)")

#start = t.time()

#df['pickup_time_30min'] = df['tpep_pickup_datetime'].dt.floor('30min')
#df['time_interval'] = df['pickup_time_30min'].dt.strftime('%H:%M') + '-' + (df['pickup_time_30min'] + pd.Timedelta(minutes=30)).dt.strftime('%H:%M')

#end = t.time()
#print(f"\nExecution time: {end - start} seconds.")


New column preview:


,tpep_pickup_datetime,tpep_pickup_time_interval
0,2020-01-01 00:28:15,00:00-00:30
1,2020-01-01 00:35:39,00:30-01:00
2,2020-01-01 00:47:41,00:30-01:00
3,2020-01-01 00:55:23,00:30-01:00
4,2020-01-01 00:09:44,00:00-00:30



List of all the possible intervals:


array(['00:00-00:30', '00:30-01:00', '23:30-00:00', '01:00-01:30',
       '21:30-22:00', '22:00-22:30', '15:30-16:00', '16:00-16:30',
       '13:30-14:00', '14:00-14:30', '23:00-23:30', '16:30-17:00',
       '19:00-19:30', '19:30-20:00', '01:30-02:00', '17:30-18:00',
       '02:00-02:30', '13:00-13:30', '02:30-03:00', '03:00-03:30',
       '03:30-04:00', '04:00-04:30', '20:00-20:30', '04:30-05:00',
       '05:00-05:30', '14:30-15:00', '18:00-18:30', '05:30-06:00',
       '06:00-06:30', '15:00-15:30', '06:30-07:00', '07:00-07:30',
       '07:30-08:00', '08:00-08:30', '17:00-17:30', '08:30-09:00',
       '09:00-09:30', '09:30-10:00', '10:00-10:30', '10:30-11:00',
       '11:00-11:30', '20:30-21:00', '11:30-12:00', '12:00-12:30',
       '12:30-13:00', '18:30-19:00', '21:00-21:30', '22:30-23:00'],
      dtype=object)




(Execution time: 21.83 seconds.)


**Method ----->	Mean comp. time requested**

astype(str).str.zfill(2) ----->	1×

.map('{:02}'.format) ----->	~0.6×

.dt.strftime('%H') su datetime ----->	3–4×

## Task 8

For each interval, determine the average number of passengers and the average fare amount

In [11]:
avg_counts = df.groupby(['tpep_pickup_time_interval'])[['passenger_count','fare_amount']].mean().round(2).reset_index()

avg_counts.columns = ['PU Time Range','Avg Passengers','Avg. Fare Amount']

print("\nAverage passenger count and average fare amount for each time interval:\n")

display(avg_counts)


Average passenger count and average fare amount for each time interval:



,PU Time Range,Avg Passengers,Avg. Fare Amount
0,00:00-00:30,1.58,13.46
1,00:30-01:00,1.59,13.11
2,01:00-01:30,1.58,12.55
3,01:30-02:00,1.59,12.17
4,02:00-02:30,1.59,12.01
5,02:30-03:00,1.59,12.01
6,03:00-03:30,1.59,12.52
7,03:30-04:00,1.59,13.04
8,04:00-04:30,1.59,14.24
9,04:30-05:00,1.52,16.38


## Task 9

For each payment type and each interval, determine the average fare amount

In [12]:
avg_fare_amount = df.groupby(['payment_type', 'tpep_pickup_time_interval'])\
                                        ['fare_amount'].mean().round(2).reset_index()

avg_fare_amount.columns = ['Payment Type', 'PU Time Range', 'Avg. Fare Amount']

print("\nAverage fare amount for each payment type and PU interval::\n")
display(avg_fare_amount)


Average fare amount for each payment type and PU interval::



,Payment Type,PU Time Range,Avg. Fare Amount
0,1.0,00:00-00:30,13.64
1,1.0,00:30-01:00,13.16
2,1.0,01:00-01:30,12.50
3,1.0,01:30-02:00,12.08
4,1.0,02:00-02:30,11.74
...,...,...,...
187,4.0,21:30-22:00,15.14
188,4.0,22:00-22:30,14.25
189,4.0,22:30-23:00,15.67
190,4.0,23:00-23:30,14.65


## Task 10

For each payment type, determine the interval when the average fare amount is maximum

In [13]:
#Using avg_fare_amount from task 9

max_avg_fare = avg_fare_amount.loc[avg_fare_amount.groupby('Payment Type')\
                                            ['Avg. Fare Amount'].idxmax()].reset_index(drop=True)

print("Most expensive pickup time range per payment type:\n")
display(max_avg_fare)

Most expensive pickup time range per payment type:



,Payment Type,PU Time Range,Avg. Fare Amount
0,1.0,05:00-05:30,20.82
1,2.0,05:00-05:30,15.03
2,3.0,04:30-05:00,16.67
3,4.0,05:00-05:30,25.19


## Task 11

For each payment type, determine the interval when the overall ratio between the tip and the fare amounts is maximum

In [14]:
df_11 = df[['payment_type', 'tpep_pickup_time_interval', 'tip_amount', 'fare_amount']]\
          .dropna(subset=['tip_amount', 'fare_amount'])\
          .query('fare_amount > 0')

print(f'Valid rows:\n{len(df_11)} out of {len(df)} - {len(df) - len(df_11)} rows removed')
print("\n\nValid payment types out of 5 (if anyone is missing, it's because it didn't have valid fare amounts):\n")
display(df_11.payment_type.unique().tolist())


tip_fare_ratio = df_11.groupby(['payment_type', 'tpep_pickup_time_interval'], as_index=False)\
                      .agg({'tip_amount': 'sum', 'fare_amount': 'sum'})\
                      .assign(tip_fare_ratio=lambda d: d['tip_amount'] / d['fare_amount'])\
                      [['payment_type', 'tpep_pickup_time_interval', 'tip_fare_ratio']]

tip_fare_ratio = tip_fare_ratio.dropna(subset=['tip_fare_ratio']).query('tip_fare_ratio > 0')


max_ratio = tip_fare_ratio.loc[tip_fare_ratio.groupby('payment_type')['tip_fare_ratio']\
                                             .idxmax()].reset_index(drop=True)

print("\n-----------------------------------\n")
print("Interval when the overall ratio between the tip and the fare amounts is maximum per payment type:\n")
display(max_ratio)

Valid rows:
6253534 out of 6254344 - 810 rows removed


Valid payment types out of 5 (if anyone is missing, it's because it didn't have valid fare amounts):



[1.0, 2.0, 4.0, 3.0]


-----------------------------------

Interval when the overall ratio between the tip and the fare amounts is maximum per payment type:



,payment_type,tpep_pickup_time_interval,tip_fare_ratio
0,1.0,18:30-19:00,0.245675
1,2.0,10:00-10:30,0.000051
2,3.0,21:00-21:30,0.005874
3,4.0,10:00-10:30,0.002012


## Task 12

Find the location with the highest average fare amount

**PLEASE NOTE:**

Since it is not specified whether the location refers to PU or DO, I'll assume the goal is to find where the most expensive trips ***begin***, and therefore I will consider the **PU** locations.

The same calculation can, of course, also be done using the DO locations, perhaps to explore the cost of reaching certain areas.

In [15]:
avg_fare_PU = df.groupby('PULocationID')['fare_amount'].mean().round(2).reset_index()

avg_fare_PU.columns =  ['location','avg_fare_amount']

res = avg_fare_PU.iloc[avg_fare_PU['avg_fare_amount'].idxmax()]

print(f"\nRegarding pickups, location {res['location']} has the highest avg. fare amount, with a total of {res['avg_fare_amount']}.\n")


Regarding pickups, location 204.0 has the highest avg. fare amount, with a total of 107.0.



## Task 13

Build a new DataFrame (called common) where, for each pickup location, we keep all trips to the 5 most common destinations (i.e., each pickup location can have different common destinations)

In [16]:
trip_frequency = df.groupby(['PULocationID', 'DOLocationID'])\
                   .size().reset_index(name='trip_freq')\
                   .sort_values(['PULocationID', 'trip_freq'], ascending=[True, False])\
                   .groupby('PULocationID', as_index=False)\
                   .head(5)

# computing frequencies PU-DO <------ .size().reset_index(name='trip_freq')
# sort values per PU->trip_freq <---- .sort_values(['PULocationID', 'trip_freq'], ascending=[True, False])\
# aggregation per PU <--------------- .groupby('PULocationID', as_index=False)\
# keep first 5 values <-------------- .head(5)

trip_frequency = trip_frequency.reset_index(drop=True)

common = df.merge(trip_frequency[['PULocationID', 'DOLocationID']], on=['PULocationID', 'DOLocationID'], how='inner')


print(f"Original DataFrame size: {len(df)}")
print(f"\nCommon DataFrame size: {len(common)}")
print(f"\nCommon is builded using {len(trip_frequency)} unique PU-DO combinations.")
print("\nFirst 5 rows of Common df:\n")
display(common.head())

Original DataFrame size: 6254344

Common DataFrame size: 1915736

Common is builded using 1243 unique PU-DO combinations.

First 5 rows of Common df:



,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,duration_sec,duration_str,tpep_pickup_time_interval
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.20,1.0,N,238,239,1.0,...,3.0,0.5,1.47,0.0,0.3,11.27,2.5,288,4 minutes 48 seconds,00:00-00:30
1,1.0,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.20,1.0,N,239,238,1.0,...,3.0,0.5,1.50,0.0,0.3,12.30,2.5,445,7 minutes 25 seconds,00:30-01:00
2,1.0,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.60,1.0,N,238,238,1.0,...,3.0,0.5,1.00,0.0,0.3,10.80,2.5,371,6 minutes 11 seconds,00:30-01:00
3,1.0,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.80,1.0,N,238,151,1.0,...,0.5,0.5,1.36,0.0,0.3,8.16,0.0,291,4 minutes 51 seconds,00:30-01:00
4,2.0,2020-01-01 00:09:44,2020-01-01 00:10:37,1.0,0.03,1.0,N,7,193,2.0,...,0.5,0.5,0.00,0.0,0.3,3.80,0.0,53,0 minutes 53 seconds,00:00-00:30


In [17]:
#OPTIONAL

pu_top_dos = trip_frequency.groupby('PULocationID')['DOLocationID']\
                           .apply(lambda x: ', '.join(map(str, x)))\
                           .reset_index(name='top_destinations')

for _, row in pu_top_dos.iterrows():
    print(f"Location {row['PULocationID']}, top DOs: {row['top_destinations']}")

Location 1, top DOs: 1, 264, 138, 265, 50
Location 2, top DOs: 56, 95, 201
Location 3, top DOs: 226, 146, 254, 3, 51
Location 4, top DOs: 79, 148, 107, 114, 249
Location 5, top DOs: 118, 14, 23, 75, 210
Location 6, top DOs: 6, 214, 115, 132, 228
Location 7, top DOs: 7, 223, 193, 179, 146
Location 8, top DOs: 237, 74, 166, 239, 262
Location 9, top DOs: 44, 9, 95, 16, 19
Location 10, top DOs: 230, 48, 170, 164, 161
Location 11, top DOs: 11, 74, 14, 26, 55
Location 12, top DOs: 230, 231, 161, 87, 48
Location 13, top DOs: 231, 246, 87, 158, 230
Location 14, top DOs: 14, 209, 228, 67, 132
Location 15, top DOs: 15, 135, 16, 265, 3
Location 16, top DOs: 16, 15, 171, 64, 28
Location 17, top DOs: 17, 217, 61, 225, 97
Location 18, top DOs: 18, 47, 254, 20, 75
Location 19, top DOs: 19, 265, 64, 101, 219
Location 20, top DOs: 20, 78, 41, 59, 136
Location 21, top DOs: 231, 21, 89, 108, 123
Location 22, top DOs: 22, 26, 14, 183, 11
Location 23, top DOs: 135, 23, 208, 15, 5
Location 24, top DOs: 238,

## Task 14

On the common DataFrame, for each payment type and each interval, determine the average fare amount

In [18]:
common_avg_fare_amount = common.groupby(['payment_type', 'tpep_pickup_time_interval'])\
                                        ['fare_amount'].mean().round(2).reset_index()

common_avg_fare_amount.columns = ['Payment Type', 'PU Time Range', 'Avg. Fare Amount']

print("\nAverage fare amount for each payment type and PU interval in the most common destinations:\n")
display(common_avg_fare_amount)


Average fare amount for each payment type and PU interval in the most common destinations:



,Payment Type,PU Time Range,Avg. Fare Amount
0,1.0,00:00-00:30,7.85
1,1.0,00:30-01:00,7.65
2,1.0,01:00-01:30,7.46
3,1.0,01:30-02:00,7.09
4,1.0,02:00-02:30,6.95
...,...,...,...
187,4.0,21:30-22:00,10.14
188,4.0,22:00-22:30,10.03
189,4.0,22:30-23:00,11.69
190,4.0,23:00-23:30,7.26


## Task 15

Compute the difference of the average fare amount computed in the previous point with those computed at point 9

In [19]:
merged_avg_fares = pd.merge(common_avg_fare_amount,avg_fare_amount,
                            on=['Payment Type', 'PU Time Range'],
                            suffixes=(' (common)', ' (general)')
                   )

merged_avg_fares['Difference'] = \
merged_avg_fares['Avg. Fare Amount (common)'] - merged_avg_fares['Avg. Fare Amount (general)']

print("\nFirst 5 difference of average fare amount between 'common' and 'general' trips for each payment type and interval:\n")
display(merged_avg_fares.head(5))

len_common = len(merged_avg_fares[merged_avg_fares['Difference']>0])
len_general = len(merged_avg_fares) - len_common
print(f"\n\nOut of the {len(merged_avg_fares)} trips into considerations:")
print(f"  -  The trips that are more expensive in the common df (task 14) are {len_common} ")
print(f"  -  The trips that are more expensive in the general df (task 9) are {len_general} ")


First 5 difference of average fare amount between 'common' and 'general' trips for each payment type and interval:



,Payment Type,PU Time Range,Avg. Fare Amount (common),Avg. Fare Amount (general),Difference
0,1.0,00:00-00:30,7.85,13.64,-5.79
1,1.0,00:30-01:00,7.65,13.16,-5.51
2,1.0,01:00-01:30,7.46,12.50,-5.04
3,1.0,01:30-02:00,7.09,12.08,-4.99
4,1.0,02:00-02:30,6.95,11.74,-4.79




Out of the 192 trips into considerations:
  -  The trips that are more expensive in the common df (task 14) are 1 
  -  The trips that are more expensive in the general df (task 9) are 191 


## Task 16

Compute the ratio between the differences computed in the previous point and those computed in point 9 Note: You have to compute a ratio for each pair (payment type, interval)

In [20]:
merged_avg_fares['Ratio'] = merged_avg_fares['Difference'] / merged_avg_fares['Avg. Fare Amount (general)']

print("\nRatio of average fare amount differences:\n")
display(merged_avg_fares[['Payment Type', 'PU Time Range', 'Difference', 'Avg. Fare Amount (general)', 'Ratio']])


Ratio of average fare amount differences:



,Payment Type,PU Time Range,Difference,Avg. Fare Amount (general),Ratio
0,1.0,00:00-00:30,-5.79,13.64,-0.424487
1,1.0,00:30-01:00,-5.51,13.16,-0.418693
2,1.0,01:00-01:30,-5.04,12.50,-0.403200
3,1.0,01:30-02:00,-4.99,12.08,-0.413079
4,1.0,02:00-02:30,-4.79,11.74,-0.408007
...,...,...,...,...,...
187,4.0,21:30-22:00,-5.00,15.14,-0.330251
188,4.0,22:00-22:30,-4.22,14.25,-0.296140
189,4.0,22:30-23:00,-3.98,15.67,-0.253989
190,4.0,23:00-23:30,-7.39,14.65,-0.504437


In [21]:
#OPTIONAL

import random

def ratio_comment(row_index):
  payment_type = merged_avg_fares.loc[row_index, 'Payment Type']
  interval = merged_avg_fares.loc[row_index, 'PU Time Range']
  ratio = merged_avg_fares.loc[row_index, 'Ratio']
  new_ratio = abs(ratio*100)
  if ratio<0:
    print(f"Common trip with payment type {payment_type} in the time range {interval} is {new_ratio:.2f}% cheaper than the general trip.")
  else:
    print(f"Common trip with payment type {payment_type} in the time range {interval} is {new_ratio:.2f}% more expensive than the general trip.")

print("Here a comment about the computed ratio of 10 random rows:\n")
for _ in range(10):
  row_index = random.randint(0, len(merged_avg_fares) - 1)
  ratio_comment(row_index)


Here a comment about the computed ratio of 10 random rows:

Common trip with payment type 1.0 in the time range 00:00-00:30 is 42.45% cheaper than the general trip.
Common trip with payment type 4.0 in the time range 14:00-14:30 is 28.92% cheaper than the general trip.
Common trip with payment type 1.0 in the time range 11:30-12:00 is 32.36% cheaper than the general trip.
Common trip with payment type 3.0 in the time range 11:00-11:30 is 18.35% cheaper than the general trip.
Common trip with payment type 4.0 in the time range 03:30-04:00 is 22.57% cheaper than the general trip.
Common trip with payment type 1.0 in the time range 05:00-05:30 is 51.68% cheaper than the general trip.
Common trip with payment type 2.0 in the time range 09:30-10:00 is 33.53% cheaper than the general trip.
Common trip with payment type 1.0 in the time range 02:30-03:00 is 42.44% cheaper than the general trip.
Common trip with payment type 4.0 in the time range 04:30-05:00 is 34.25% cheaper than the general t

## Task 17

Build chains of trips. Two trips are consecutive in a chain if:


*   They have the same VendorID
*   The pickup location of the second trip is also the dropoff location of the first trip
*   The pickup time of the second trip is after the dropoff time of the first trip
*   The pickup time of the second trip is at most 2 minutes later than the dropoff time of the first trip

**REASONING**

Task preparation:

- reorder the dataset by VendorID, tpep_pickup_datetime, tpep_dropoff_datetime

- create a column ‘starting_dropoff’ that contains tpep_dropoff_datetime + 1 second

- create a column ‘ending_dropoff’ that contains tpep_dropoff_datetime + 2 minutes

- initialize n = 0

Then define a main function that:

1. creates a column chain initialized to n

2. increments n by 1 and takes the first row “i” with chain = 0 and does the following:

  - chain of i = n

  - crea te a subset of rows whose tpep_pickup_datetime is between the starting_dropoff of i and the ending_dropoff of i and have the same VendorID as i

  - check whether in this subset, in the PULocationID column, the DOLocationID of i appears

  - if it does not appear: go to step 3

  - if it does appear: this row is called “j”, chain of j becomes n, and repeat steps 3A to 3E for j (which thus becomes the new i) but without changing n

3. move to the next row with chain = 0 (the new “i”), increment n by 1 and repeat steps 3A to 3E for the new row